In [8]:
import geopandas as gpd
import pandas as pd

In [3]:
hgd = gpd.read_file('data/HangJeongDong_ver20190908.geojson')

In [5]:
hgd.head()

,OBJECTID,adm_nm,adm_cd,adm_cd2,geometry
0,1,서울특별시 종로구 사직동,1101053,1111053000,(POLYGON ((126.9768888427482 37.57565077944879...
1,2,서울특별시 종로구 삼청동,1101054,1111054000,"(POLYGON ((126.982689386493 37.5950655194224, ..."
2,3,서울특별시 종로구 부암동,1101055,1111055000,(POLYGON ((126.9758511377569 37.59656422224408...
3,4,서울특별시 종로구 평창동,1101056,1111056000,(POLYGON ((126.9750746678809 37.63138628651299...
4,5,서울특별시 종로구 무악동,1101057,1111057000,"(POLYGON ((126.960673532739 37.58079784202972,..."


In [6]:
hgd['adm_cd2'] = hgd['adm_cd2'].str[:-2]
hgd['adm_cd2'] = hgd['adm_cd2'].astype('int')
hgd = hgd.rename(columns={'adm_cd2': '행정동코드'})

In [9]:
hgd = pd.concat([
    hgd,
    hgd['adm_nm'].str.split(expand=True).rename(columns={0:'시', 1:'구', 2:'동'})
], axis=1)

In [10]:
hgd = hgd[hgd['시'] == '서울특별시'][['시', '구', '동', '행정동코드', 'geometry']]

In [11]:
hgd.head()

,시,구,동,행정동코드,geometry
0,서울특별시,종로구,사직동,11110530,(POLYGON ((126.9768888427482 37.57565077944879...
1,서울특별시,종로구,삼청동,11110540,"(POLYGON ((126.982689386493 37.5950655194224, ..."
2,서울특별시,종로구,부암동,11110550,(POLYGON ((126.9758511377569 37.59656422224408...
3,서울특별시,종로구,평창동,11110560,(POLYGON ((126.9750746678809 37.63138628651299...
4,서울특별시,종로구,무악동,11110570,"(POLYGON ((126.960673532739 37.58079784202972,..."


In [12]:
hgd.shape

(424, 5)

In [13]:
hgd.to_file('data/seoul-dong.geojson', driver='GeoJSON')

---

In [14]:
import geopandas as gpd

seoul = gpd.read_file('data/seoul-dong.geojson')

In [15]:
seoul.head()

,시,구,동,행정동코드,geometry
0,서울특별시,종로구,사직동,11110530,(POLYGON ((126.9768888427482 37.57565077944879...
1,서울특별시,종로구,삼청동,11110540,"(POLYGON ((126.982689386493 37.5950655194224, ..."
2,서울특별시,종로구,부암동,11110550,(POLYGON ((126.9758511377569 37.59656422224408...
3,서울특별시,종로구,평창동,11110560,(POLYGON ((126.9750746678809 37.63138628651299...
4,서울특별시,종로구,무악동,11110570,"(POLYGON ((126.960673532739 37.58079784202972,..."


In [16]:
seoul.shape

(424, 5)

In [67]:
# 서울시 고령자현황 (동별) 통계
# https://data.seoul.go.kr/dataList/datasetView.do?infId=10020&srvType=S&serviceKind=2

pop = pd.read_csv('data/older_population_org.csv', delimiter='\t')

In [68]:
pop.head()

,기간,자치구,동,전체인구,전체인구.1,전체인구.2,65세이상 인구,65세이상 인구.1,65세이상 인구.2,65세이상 인구.3,65세이상 인구.4,65세이상 인구.5,65세이상 인구.6,65세이상 인구.7,65세이상 인구.8
0,기간,자치구,동,전체인구,전체인구,전체인구,합계,합계,합계,내국인,내국인,내국인,외국인,외국인,외국인
1,기간,자치구,동,계,남자,여자,계,남자,여자,계,남자,여자,계,남자,여자
2,2019.3/4,합계,합계,"10,025,927","4,887,068","5,138,859","1,468,146","648,586","819,560","1,461,460","645,055","816,405","6,686","3,531","3,155"
3,2019.3/4,종로구,소계,"162,860","78,523","84,337","27,394","12,156","15,238","27,181","12,034","15,147",213,122,91
4,2019.3/4,종로구,사직동,"9,700","4,375","5,325","1,773",764,"1,009","1,762",758,"1,004",11,6,5


In [69]:
pop = pop[['자치구', '동', '전체인구', '전체인구.1', '전체인구.2']].rename(columns={
    '자치구': '구',
    '전체인구': '인구',
    '전체인구.1': '남자',
    '전체인구.2': '여자'
})

pop = pop[~pop['동'].isin(['동', '합계', '소계'])]

In [70]:
# 이름 맞춰줘야함...
seoul['동'] = seoul['동'].str.replace('·', '.')
pop['동'] = pop['동'].str.replace('·', '.')

In [76]:
# 숫자 형으로 변환
pop[['인구', '남자', '여자']] = pop[['인구', '남자', '여자']].applymap(lambda x : x.replace(',', ''))
pop[['인구', '남자', '여자']] = pop[['인구', '남자', '여자']].astype('int')

In [77]:
pop.head()

,구,동,인구,남자,여자
4,종로구,사직동,9700,4375,5325
5,종로구,삼청동,3013,1443,1570
6,종로구,부암동,10525,5002,5523
7,종로구,평창동,18830,8817,10013
8,종로구,무악동,8745,4078,4667


In [78]:
pop.to_csv('data/older_population.csv', index=False)

In [66]:
seoul.to_file('data/seoul-dong.geojson', driver='GeoJSON')

In [79]:
df = pd.merge(seoul[['시', '구', '동', '행정동코드', 'geometry']], 
              pop,
              how='left')
df.head()

,시,구,동,행정동코드,geometry,인구,남자,여자
0,서울특별시,종로구,사직동,11110530,(POLYGON ((126.9768888427482 37.57565077944879...,9700,4375,5325
1,서울특별시,종로구,삼청동,11110540,"(POLYGON ((126.982689386493 37.5950655194224, ...",3013,1443,1570
2,서울특별시,종로구,부암동,11110550,(POLYGON ((126.9758511377569 37.59656422224408...,10525,5002,5523
3,서울특별시,종로구,평창동,11110560,(POLYGON ((126.9750746678809 37.63138628651299...,18830,8817,10013
4,서울특별시,종로구,무악동,11110570,"(POLYGON ((126.960673532739 37.58079784202972,...",8745,4078,4667


In [80]:
df.shape

(424, 8)

In [81]:
df.to_file('data/older_seoul.geojson', driver='GeoJSON')

---

In [82]:
df = gpd.read_file('data/culture_facility_seoul.csv')

In [83]:
df.head()

,번호,주제분류,문화시설명,주소,X좌표,Y좌표,전화번호,팩스번호,홈페이지,관람시간,...,시설소개,무료구분,지하철,버스정거장,YELLOW,GREEN,BLUE,RED,공항버스,geometry
0,1,기타,간송미술문화재단,서울특별시 중구 을지로 281,37.56741127787706,127.0108890181753,010-6276-8814,,ddp.or.kr,,...,<div class=&quot;brd_viewer program_dtview tab...,,동대문역사문화공원역 1번 출구(2 4 5호선) DDP건물 살림터 3층,,,,,,,None
1,404,도서관,강동구립강일도서관,서울 강동구 강일동 678,37.5649594,127.1716731,02-427-4004,,http://www.gdlibrary.or.kr/gilib/main.do,,...,,,,,,,,,,None
2,199,도서관,강남구립못골한옥어린이도서관,서울특별시 강남구 자곡로7길 3,37.4733049813,127.099727635,02-2226-5930,,http://hchildlib.gangnam.go.kr/,,...,<div><br />\n \n </div><div>순수&nbsp;한옥도서관</div...,,수서역,못골마을 LH강남아이파크 정류장,,,,,,None
3,5,기타,조계사,서울특별시 종로구 우정국로 55,37.573893952230144,126.9818832690818,02-768-7543,,http://www.jogyesa.kr/,,...,서울시 종로구에 소재한 대한불교조계종 총본산 조계사입니다.,,1호선 종각역 2번출구 3호선 안국역 6번 출구 광화문역 2번 출구,조계사 정거장 안국동 정거장,,안국동 정거장 : 1020번,조계사 정거장 : 109번 151번 162번 172번 606번 안국동 정거장 :...,인사동 정거장 : 6005번,6005번,None
4,200,도서관,강남구립열린도서관,서울특별시 강남구 학동로 343 포바강남타워 B2,37.5174497661,127.0406357728,02-6004-3970,,http://yllib.gangnam.go.kr/,,...,,,,,,,,,,None


In [10]:
import pandas as pd

toilet = pd.read_csv('data/toilet_seoul.csv')
toilet.head()

,고유번호,구명,법정동명,산지여부,주지번,부지번,새주소명,생성일,위도,경도
0,92,송파구,마천동,2,1,10,,NaN,37.501401,127.158647
1,93,노원구,하계동,1,288,2,,NaN,37.644937,127.073728
2,94,노원구,하계동,1,255,,,NaN,37.640335,127.077637
3,96,강북구,수유동,1,252,19,,NaN,37.645723,127.020449
4,98,도봉구,방학동,1,270,1,,NaN,37.659783,127.031187


In [11]:
import numpy as np
np.random.randint(0, 500, size=len(toilet))

array([156, 148, 139, ..., 278, 191, 469])

In [14]:
toilet['이용량'] = np.random.randint(0, 500, size=len(toilet))
toilet = toilet[['구명', '법정동명', '이용량', '위도', '경도']]

In [15]:
toilet.head(3)

,구명,법정동명,이용량,위도,경도
0,송파구,마천동,248,37.501401,127.158647
1,노원구,하계동,252,37.644937,127.073728
2,노원구,하계동,192,37.640335,127.077637


In [17]:
from mapboxgl.utils import df_to_geojson

df_to_geojson(toilet, filename='data/toilet_seoul_sample.geojson',
              properties=['구명', '법정동명', '이용량'],
              lat='위도', lon='경도', precision=6,)

{'type': 'file',
 'filename': 'toilet_seoul_sample.geojson',
 'feature_count': 5046}

---

In [12]:
import pandas as pd

df = pd.read_csv('data/seoul_road_network.csv')
df.head()

/Users/heumsi/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,순서,시군구코드,도로구간일련번호,도로명,도로명코드,영문도로명,고시일자,도로명심의일자,도로제정권자,광역도로구분코드,...,이동사유코드,이동사유,이동일자,분기시군구코드,분기도로구간일련번호,입력방법,교차로수,발급여부,위도,경도
0,1,11110,3226,새문안로,3005004,Saemunan-ro,20100422,20100412,시도,2,...,71,도로명기본도위치정확도개선사업에 따른 기재정정,,,0,1,0,,37.570884,126.973331
1,2,11110,3227,옥인6가길,4100210,Ogin 6ga-gil,20100702,20100623,시군구,3,...,71,도로명기본도위치정확도개선사업에 따른 기재정정,,,0,1,0,,37.581420,126.965176
2,3,11110,3229,자하문로,3100012,Jahamun-ro,20100702,20100623,시군구,3,...,71,도로명기본도위치정확도개선사업에 따른 기재정정,,,0,1,0,,37.586980,126.969095
3,4,11110,1818,삼일대로17길,4100140,Samil-daero 17-gil,20100702,20100623,시군구,3,...,71,도로명기본도위치정확도개선사업에 따른 기재정정,,,0,1,0,,37.569189,126.986074
4,5,11110,2860,사직로1길,4100126,Sajik-ro 1-gil,20100702,20100623,시군구,3,...,71,도로명기본도위치정확도개선사업에 따른 기재정정,,,0,1,0,,37.573648,126.962948


In [13]:
df.shape

(71040, 30)

In [14]:
df.iloc[0]

순서                                   1
시군구코드                            11110
도로구간일련번호                          3226
도로명                               새문안로
도로명코드                          3005004
영문도로명                      Saemunan-ro
고시일자                          20100422
도로명심의일자                       20100412
도로제정권자                              시도
광역도로구분코드                             2
도로위계기능구분                             3
도로구간종속구분                             1
기점                          신문로1가 74-2
종점                           당주동 100-5
도로폭                                 40
도로길이                               177
기초간격                                20
국도/지방도번호                              
부여사유            서대문안을 의미하는 새문안의 의미를 인용
부여일자                          20100419
이동사유코드                              71
이동사유          도로명기본도위치정확도개선사업에 따른 기재정정
이동일자                                  
분기시군구코드                               
분기도로구간일련번호                           0
입력방법                     

In [34]:
import geopandas as gpd

df = gpd.read_file('data/seoul_road_network/TL_SPRD_MANAGE_W.shp')

In [42]:
df['SIG_CD'].value_counts()

11290    5156
11260    4168
11305    3976
11620    3963
11230    3686
11440    3555
11560    3484
11110    3478
11410    3438
11170    3230
11215    2977
11530    2686
11200    2532
11500    2525
11380    2485
11590    2373
11710    2228
11650    2163
11350    2162
11320    2138
11140    2003
11545    1861
11680    1835
11740    1756
11470    1182
Name: SIG_CD, dtype: int64

In [55]:
# 11590 : 여의도
df = df[df['SIG_CD'] == '11590']

In [56]:
df = df.rename(columns = {
    'RN' : '도로명',
    'ROAD_PY_LT' : '도로길이',
    'ROAD_BT': '도로폭',
})[['도로명', '도로길이', '도로폭', 'geometry']]
df.head()

,도로명,도로길이,도로폭,geometry
52380,만양로8길,53,7,"LINESTRING (126.947776365 37.50835655899999, 1..."
52381,동작대로39가길,450,5,"LINESTRING (126.9809748709999 37.493643061, 12..."
52382,등용로6길,65,4,"LINESTRING (126.933448546 37.50607434300002, 1..."
52436,여의대방로10가길,249,6,"LINESTRING (126.914844826 37.49345082500002, 1..."
52570,국사봉1나길,147,4,"LINESTRING (126.934785104 37.49486913700002, 1..."


In [57]:
df.shape

(2373, 4)

In [60]:
df['도로폭'].value_counts().sort_index()

0     105
1       6
2     235
3     285
4     428
5     260
6     316
7     279
8     140
9      67
10     24
11     33
12     62
13     30
14      5
15     47
16     15
17      5
19      1
20      7
21      8
25      2
30      6
35      3
40      2
50      2
Name: 도로폭, dtype: int64

In [58]:
df.to_file('data/yeouido_road.geojson', driver='GeoJSON')